# Import Packages

In [1]:
from decouple import config
import time

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

import google.generativeai as genai

API_KEY = config("API_KEY")
genai.configure(api_key=API_KEY)

/Users/johnny_hsieh/anaconda3/envs/rag_gemini/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Set parameters

In [2]:
# Sentence embedding
EMBED_MODEL_NAME = "DMetaSoul/sbert-chinese-general-v2"

# Gemini Settings
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]

generation_config = {
  "temperature": 0.9,
  "max_output_tokens": 512,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
    "gemini-pro", 
    safety_settings, 
    generation_config=generation_config
    )

# Function for loading vector DB and generating a retriever

In [3]:
def generate_retriever():
    print("Loading vector DB...")
    # Change to "cpu" or "cuda" if your machine is Windows.
    model_kwargs = {"device": "mps"}
    embedding = HuggingFaceEmbeddings(model_name=EMBED_MODEL_NAME, model_kwargs=model_kwargs)
    
    db = FAISS.load_local("../diabetic_vector_db", embedding, allow_dangerous_deserialization=True)
    print("Done loading vector DB!\n")
    return db.as_retriever(search_kwargs={"k": 5})

# Function for searching the related content

In [4]:
def search_related_content(retriever, query):
    docs = retriever.invoke(query)
    related_context = "\n---\n".join([doc.page_content for doc in docs])
    return related_context

# Function for generating the answer

In [5]:
def generate_answer(query, related_context):
    template = f"""
任務: 
1. 你是一位在台灣的糖尿病領域的專業護理師，需要以親切的口吻回答病患的問題。
2. 你必須依照下方的「相關文本」，再透過同義的改寫生成出最後的答案。
3. 輸出限制： 最多60字、只能使用繁體中文、純文字格式
4. 如果「相關文本」中有答案，一定要引用「相關文本」來回答；如果判斷與「病患的提問」沒有關連只要回答「不好意思，我不清楚。」即可。
------
「相關文本」：
{related_context}
------
「病患的提問」：
{query}
"""
    print(template)
    start_time = time.time()
    response = model.generate_content(template)
    qa_result = response.text
    end_time = time.time()
    print(f"Execution Time: {end_time - start_time}")
    return qa_result

# Generate retriever

In [6]:
retriever = generate_retriever()

Loading vector DB...


/Users/johnny_hsieh/anaconda3/envs/rag_gemini/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Done loading vector DB!



# Search related content and generate answer

In [7]:
question = ["為什麼在飲食控制的過程中休息也很重要？", "哪些是需要注意的常見問題？", "如何提高飲食記錄的回顧效率？", "氣喘如何治療？"]

answers = []
for query in question:
    related_context = search_related_content(retriever, query)
    result = generate_answer(query, related_context)
    answers.append(result)
    time.sleep(5)


任務: 
1. 你是一位在台灣的糖尿病領域的專業護理師，需要以親切的口吻回答病患的問題。
2. 你必須依照下方的「相關文本」，再透過同義的改寫生成出最後的答案。
3. 輸出限制： 最多60字、只能使用繁體中文、純文字格式
4. 如果「相關文本」中有答案，一定要引用「相關文本」來回答；如果判斷與「病患的提問」沒有關連只要回答「不好意思，我不清楚。」即可。
------
「相關文本」：
記錄飲食一段時間後，會發現某些食物經常重複出現。這時會推薦你將這些常出現的食物，整理成自己常吃的食物選項。同樣是智抗糖App內的功能：你可以將常吃的食物加入”最愛”選項，如此一來就能一鍵記錄食物了。除此之外，也可以透過這個方式，判斷自己常吃的食物的頻率、份量，了解對血糖、體重及其他健康指標的影響。分享了這麼多，其實也不希望帶給糖友們壓力。在記錄飲食日記時，多多少少也會感到疲勞，或是偶爾真的想休息一下的時候。這時也可以安排一些喘息空間，例如週一到週五詳細記錄，週六日以拍照為主；或者，真的沒記錄到也無妨，適時讓自己放個假，在飲食控制的過程中稍做休息也是十分重要的喔。
---
腿熱量就相差約200大卡，如果只記錄了雞腿便當，便難以判斷體重變化和這一餐的關係。最後，加工過程也會影響到食物的營養素含量，確實是比較難記錄的食物類型 。 如果有營養標示，可以優先參考並記錄份量和營養素比例。而小吃 、火鍋料等沒有包裝的加工食品，則建議先拍照或用文字簡單記下份量，回診時與營養師討論，請專業的醫護人員，透過經驗來評估對血糖的影響程度。第四，善用科技工具，降低記錄難度。以智抗糖App為例，飲食記錄方面提供了多種功能，包括拍攝食物、上傳照片、使用食物資料庫、掃描包裝條碼以及拍攝照片並透過AI辨識等。在食物資料庫功能下，只需輸入關鍵字即可快速查詢到相應的食物選項，並立即記錄相關的營養素，有效節省了打字、查資料和拍照的時間
---
在記錄飲食時，我們可以選擇不同的方法，每種方法都有其優點和缺點。文字記錄的優點是有高度的自由度，能夠進行詳細的描述，但相對地，缺點是獲取食物資訊可能較為困難，且可能耗費較多時間。拍照記錄的優點是快速方便，能夠完整地記錄飲食情況，然而，缺點是僅僅依靠照片可能難以辨識食物的份量，並且難以判斷食物的種類。資料庫搜尋的優勢在於可以即時獲取食物的營養組成 ， 便於進行飲食調整，但缺點是食物選擇較

In [8]:
for q, a in zip(question,answers):
    print(q, a)

為什麼在飲食控制的過程中休息也很重要？ 適時讓自己休息，在飲食控制過程中稍做放鬆也很重要喔！
哪些是需要注意的常見問題？ 不好意思，我不清楚。
如何提高飲食記錄的回顧效率？ 記錄關鍵資訊，比如醣量和熱量。透過拍照、資料庫查詢，讓飲食記錄更完整。
氣喘如何治療？ 不好意思，我不清楚。
